# Interactions - Lab

## Introduction

In this lab, you'll explore interactions in the Boston Housing data set.

## Objectives

You will be able to:
- Understand what interactions are
- Understand how to accommodate for interactions in regression

## Build a baseline model 

You'll use a couple of built-in functions, which we imported for you below.

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Import the Boston data set using `load_boston()`. We won't bother to preprocess the data in this lab. If you still want to build a model in the end, you can do that, but this lab will just focus on finding meaningful insights in interactions and how they can improve $R^2$ values.

In [2]:
regression = LinearRegression()
boston = load_boston()

Create a baseline model which includes all the variables in the Boston housing data set to predict the house prices. Then use 10-fold cross-validation and report the mean $R^2$ value as the baseline $R^2$.

In [15]:
## code here
regression.fit(boston.data, boston.target)

crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)

baseline = np.mean(cross_val_score(regression, boston.data, boston.target, scoring="r2", cv=crossvalidation))

In [16]:
baseline

0.7190106820189475

## See how interactions improve your baseline

Next, create all possible combinations of interactions, loop over them and add them to the baseline model one by one to see how they affect the $R^2$. We'll look at the 3 interactions which have the biggest effect on our $R^2$, so print out the top 3 combinations.

You will create a for loop to loop through all the combinations of 2 predictors. You can use `combinations` from itertools to create a list of all the pairwise combinations. To find more info on how this is done, have a look [here](https://docs.python.org/2/library/itertools.html).

In [17]:
from itertools import combinations
combinations = list(combinations(boston.feature_names, 2))

In [31]:
y = pd.DataFrame(boston.target,columns = ["target"])
df = pd.DataFrame(boston.data, columns = boston.feature_names)
all_data = pd.concat([y,df], axis = 1)
all_data.head()

,target,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,24.0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,21.6,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,34.7,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,33.4,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,36.2,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [33]:
r2_values = []
for combo in combinations:
    temp = pd.DataFrame(boston.data, columns = boston.feature_names)
    temp['inter'] = temp[combo[0]]*temp[combo[1]]
    regression.fit(temp, all_data['target'])
    crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
    r2 = np.mean(cross_val_score(regression, boston.data, boston.target, scoring="r2", cv=crossvalidation))
    r2_values.append((combo,r2))

In [34]:
## code to find top 3 interactions by R^2 value here
r2_values

[(('CRIM', 'ZN'), 0.7190106820189475),
 (('CRIM', 'INDUS'), 0.7190106820189475),
 (('CRIM', 'CHAS'), 0.7190106820189475),
 (('CRIM', 'NOX'), 0.7190106820189475),
 (('CRIM', 'RM'), 0.7190106820189475),
 (('CRIM', 'AGE'), 0.7190106820189475),
 (('CRIM', 'DIS'), 0.7190106820189475),
 (('CRIM', 'RAD'), 0.7190106820189475),
 (('CRIM', 'TAX'), 0.7190106820189475),
 (('CRIM', 'PTRATIO'), 0.7190106820189475),
 (('CRIM', 'B'), 0.7190106820189475),
 (('CRIM', 'LSTAT'), 0.7190106820189475),
 (('ZN', 'INDUS'), 0.7190106820189475),
 (('ZN', 'CHAS'), 0.7190106820189475),
 (('ZN', 'NOX'), 0.7190106820189475),
 (('ZN', 'RM'), 0.7190106820189475),
 (('ZN', 'AGE'), 0.7190106820189475),
 (('ZN', 'DIS'), 0.7190106820189475),
 (('ZN', 'RAD'), 0.7190106820189475),
 (('ZN', 'TAX'), 0.7190106820189475),
 (('ZN', 'PTRATIO'), 0.7190106820189475),
 (('ZN', 'B'), 0.7190106820189475),
 (('ZN', 'LSTAT'), 0.7190106820189475),
 (('INDUS', 'CHAS'), 0.7190106820189475),
 (('INDUS', 'NOX'), 0.7190106820189475),
 (('INDU

## Look at the top 3 interactions: "RM" as a confounding factor

The top three interactions seem to involve "RM", the number of rooms as a confounding variable for all of them. Let's have a look at interaction plots for all three of them. This exercise will involve:

- splitting the data up in 3 groups: one for houses with a few rooms, one for houses with a "medium" amount of rooms, one for a high amount of rooms.
- Create a function `build_interaction_rm`. This function takes an argument `varname` (which can be set equal to the column name as a string) and a column `description` (which describes the variable or varname, to be included on the x-axis of the plot). The function outputs a plot that uses "RM" as a confounding factor. Each plot should have three regression lines, one for each level of "RM." 

The data has been split into high, medium and low number of rooms for you.

In [ ]:
rm = np.asarray(df[["RM"]]).reshape(len(df[["RM"]]))

In [ ]:
high_rm = all_data[rm > np.percentile(rm, 67)]
med_rm = all_data[(rm > np.percentile(rm, 33)) & (rm <= np.percentile(rm, 67))]
low_rm = all_data[rm <= np.percentile(rm, 33)]

Create `build_interaction_rm`.

In [ ]:
def build_interaction_rm(varname, description):
    pass

Next, use build_interaction_rm with the three variables that came out with the highest effect on $R^2$

In [ ]:
# first plot

In [ ]:
# second plot

In [ ]:
# third plot

## Build a final model including all three interactions at once

Use 10-fold cross validation.

In [ ]:
# code here

In [ ]:
# code here

Our $R^2$ has increased considerably! Let's have a look in statsmodels to see if all these interactions are significant.

In [ ]:
# code here

What is your conclusion here?

In [ ]:
# formulate your conclusion

## Summary

You should now understand how to include interaction effects in your model! As you can see, interactions can have a strong impact on linear regression models, and they should always be considered when you are constructing your models.